In [177]:
import pandas as pd
import numpy as np
import math
from numpy import dot
from sklearn.metrics import accuracy_score

<b> Q1.1 (20%) Download the Adult dataset. Clean up the dataset and create x_train, y_train, x_test, y_test for training feature, training value, test feature, test label. All of these variables should be numpy arrays. Provide summary statistics for your training and test datasets so that TA can verify the correctness of your procedure.

* 資料匯入

In [2]:
Column = ["age","workclass","fnlwgt","education","education-num",
          "marital-status","occupation","relationship","race","sex", "capital-gain",
          "capital-loss","hours-per-week","native-country", "income"]
train = pd.read_csv("adult_data.csv", index_col = False, names = Column)
test = pd.read_csv("adult_test.csv", index_col = False, names = Column)
test = test.drop(test.index[0])

In [3]:
# 處理缺失值，並把index重設
train = train[train[Column] != " ?"].dropna()
train = train.reset_index(drop = True)

test = test[test[Column] != " ?"].dropna()
test = test.reset_index(drop = True)

//anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


* 此時還需要注意，待會我們會進行Nominal Variable轉換成Dummy作為Model Feature的步驟，而依照題目指示，我們只要出現超過10次的Variable，通過train.iloc[;, n].value_counts( ) for n in [1,3,5,6,7,8,9,13]可得知，需要Drop掉的Nominal Variables有：
    * Occupation: Armed-Forces
    * Native-Country: Holand-Netherlands

In [7]:
Armed_Forces_train = train[train["occupation"] == " Armed-Forces"].index
Holand_Netherlands_train = train[train["native-country"] == " Holand-Netherlands"].index
Armed_Forces_test = test[test["occupation"] == " Armed-Forces"].index

train.drop(Armed_Forces_train, inplace = True)
train.drop(Holand_Netherlands_train, inplace = True)
test.drop(Armed_Forces_test, inplace = True)

In [8]:
# 把test set的數值定為integer，與training set一致
test["age"] = test["age"].astype(int)
test["fnlwgt"] = test["fnlwgt"].astype(int)
test["education-num"] = test["education-num"].astype(int)
test["capital-gain"] = test["capital-gain"].astype(int)
test["capital-loss"] = test["capital-loss"].astype(int)
test["hours-per-week"] = test["hours-per-week"].astype(int)

In [9]:
# 名目資料轉DUMMIES
train = pd.get_dummies(train, columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country", "income"], drop_first = True)
test = pd.get_dummies(test, columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country", "income"], drop_first = True)

* Feature-Label Split

In [16]:
x_train = train.iloc[:, :94].values
y_train = train.iloc[:, 94].values
x_test = test.iloc[:, :94].values
y_test = test.iloc[:, 94].values

In [189]:
x_train

array([[    39,  77516,     13, ...,      1,      0,      0],
       [    50,  83311,     13, ...,      1,      0,      0],
       [    38, 215646,      9, ...,      1,      0,      0],
       ...,
       [    58, 151910,      9, ...,      1,      0,      0],
       [    22, 201490,      9, ...,      1,      0,      0],
       [    52, 287927,      9, ...,      1,      0,      0]])

In [190]:
y_train

array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)

In [191]:
x_test

array([[    25, 226802,      7, ...,      1,      0,      0],
       [    38,  89814,      9, ...,      1,      0,      0],
       [    28, 336951,     12, ...,      1,      0,      0],
       ...,
       [    38, 374983,     13, ...,      1,      0,      0],
       [    44,  83891,     13, ...,      1,      0,      0],
       [    35, 182148,     13, ...,      1,      0,      0]])

In [192]:
y_test

array([0, 0, 1, ..., 0, 0, 1], dtype=uint8)

* Summary statistics of features and labels

In [22]:
# MEAN OF CONTINUOUS VARIABLES
train_describe = [len(train)]
for i in range(6):
    sum_train= 0
    for j in range(len(train)):
        sum_train += x_train[j][i]
    train_describe.append(sum_train/len(x_train))
    
test_describe = [len(test)]
for i in range(6):
    sum_test = 0
    for j in range(len(test)):
        sum_test += x_test[j][i]
    test_describe.append(sum_test/len(x_test))
    
pd.DataFrame([train_describe, test_describe], index = ["Training", "Testing"], columns = ["Count", 'Age', 'FnlWgt', 'Education-Num', 'Capital-Gain', 'Capital-Loss',
       'Hours-Per-Week'])

,Count,Age,FnlWgt,Education-Num,Capital-Gain,Capital-Loss,Hours-Per-Week
Training,30152,38.440568,189791.552932,10.121319,1092.370025,88.266085,40.931348
Testing,15055,38.769711,189604.611956,10.112255,1120.188907,89.071471,40.950714


In [23]:
# Label Counts
from collections import Counter
train_count = Counter(y_train)
test_count = Counter(y_test)
pd.DataFrame(data = [train_count.values(), test_count.values()], index = ["Training", "Testing"], columns = ["<= 50k", "> 50k"])

,<= 50k,> 50k
Training,22645,7507
Testing,11358,3697


<b> Q1.2 (20%) Derive the gradient and hessian matrix for the new E(w).

* Lambda Vector

In [212]:
lambda_vec1 = np.ones(x_train.shape[1] + 1)
lambda_vec2 = np.hstack((np.ones(x_train.shape[1]), 0))
lambda_vec3 = np.hstack((np.ones(5), np.array([0.5] * 89), 0))

* 初始Feature Weight，使用公式𝑤 = (𝑋<sup>𝑇</sup>𝑋+𝑏𝐼)<sup>−1</sup>𝑋<sup>𝑇</sup>𝑡

In [194]:
w = np.dot(np.linalg.inv(np.dot(x_train.T, x_train) + lambda_vec1.mean() * np.eye(x_train.shape[1])), np.dot(x_train.T, y_train))

In [195]:
w

array([ 2.76285968e-03,  7.95062972e-08, -2.75693956e-03,  7.70476784e-06,
        9.11983024e-05,  2.83119772e-03, -1.04182437e-01, -7.21730205e-02,
       -1.66026332e-02, -1.38927468e-01, -1.18296027e-01, -2.17496320e-01,
        2.84319210e-02,  5.37060012e-02, -5.37704550e-02, -4.56289945e-02,
       -8.42049396e-02, -4.27561383e-02,  1.16528858e-01,  1.12520549e-01,
        2.17335354e-01,  4.19439167e-01,  4.45101182e-02,  3.07537415e-01,
       -3.62323913e-02,  3.76434042e-01,  9.31072233e-02,  1.89943178e-01,
        9.93077142e-02,  3.05254783e-02, -9.91639598e-03,  1.21189483e-02,
        1.62700799e-02, -2.17451430e-02,  1.25628263e-01, -1.08005070e-01,
       -5.88570979e-02, -5.03328174e-02, -2.40657697e-02,  1.34223469e-02,
        6.27060244e-02,  7.05143173e-02,  3.91081086e-02,  6.80940383e-02,
       -4.47886986e-02, -1.78708546e-01, -1.47180592e-01, -1.53660589e-01,
       -1.64257747e-01,  1.23375969e-01,  4.06891022e-02,  2.09184028e-02,
       -6.77090778e-03,  

* 用Sigmoid Function得出給定X-Feature為C1_Class的機率為何

In [196]:
def sigmoid(alpha):
    return 1 / (1 + np.exp(-alpha))

* Gradient
    -  ∇E(ω) = Φ<sup>T</sup> · (y-t)

In [218]:
def Gradient(x, y, w):
    alpha = np.dot(x, w)
    t = sigmoid(alpha)
    return np.dot(x.T, (y-t))

gradient = Gradient(x_train, y_train, w)
gradient

array([-3.29876918e+05, -1.79311471e+09, -8.66428459e+04,  6.47336680e+06,
       -2.27054366e+05, -3.57480112e+05, -5.73333200e+02, -7.45744231e+03,
       -8.02030576e+01, -7.09634854e+02, -3.79344231e+02, -7.05449477e+00,
       -4.79525881e+02, -1.66134806e+02, -7.09982384e+01, -1.34977894e+02,
       -2.52202356e+02, -2.08691182e+02, -3.10914359e+02, -3.94149543e+02,
       -9.12594928e+02,  2.65221933e+01, -3.69982539e+03, -1.16621060e+02,
       -2.25067504e+01,  4.00253609e+01, -2.32997432e+03, -2.89541799e+00,
       -2.18841601e+03, -1.61627473e+02, -4.50558986e+03, -4.19789176e+02,
       -3.53322924e+02, -1.33130914e+03, -5.27158645e+02, -4.07961954e+02,
       -6.12503614e+02, -7.98133485e+02, -1.50639536e+03, -7.07339496e+01,
       -6.44551233e+02, -1.63586457e+02, -1.05895397e+03, -2.46175075e+02,
       -5.45906762e+02, -3.24142680e+03, -4.16690207e+02, -2.18431057e+03,
       -1.44574273e+03, -1.77673131e+02, -2.59767862e+02, -1.13179728e+03,
       -9.95575588e+01, -

* Hessian
    - ∇E(ω) = Φ<sup>T</sup> · R · Φ

In [198]:
def Hessian(x, w):
    t = sigmoid(np.dot(x, w))
    R = np.diag(t * (1 - t))
    return np.dot(np.dot(x.T, R), x)
    
hessian = Hessian(x_train, w)
hessian

array([[1.19604528e+07, 5.23008304e+10, 2.82216517e+06, ...,
        2.55288566e+05, 5.25079104e+02, 1.48045027e+02],
       [5.23008304e+10, 3.45056407e+14, 1.38714402e+10, ...,
        1.24594057e+09, 2.60837022e+06, 8.20864146e+05],
       [2.82216517e+06, 1.38714402e+10, 7.86904187e+05, ...,
        6.77254585e+04, 1.49773881e+02, 3.77258001e+01],
       ...,
       [2.55288566e+05, 1.24594057e+09, 6.77254585e+04, ...,
        6.66116297e+03, 0.00000000e+00, 0.00000000e+00],
       [5.25079104e+02, 2.60837022e+06, 1.49773881e+02, ...,
        0.00000000e+00, 1.57839717e+01, 0.00000000e+00],
       [1.48045027e+02, 8.20864146e+05, 3.77258001e+01, ...,
        0.00000000e+00, 0.00000000e+00, 3.82039347e+00]])

<b>Q1.3 (30%) Create your mylogistic_l2 class. Show the learned 𝑤 as well as test accuracy for the cases below. If 𝑤 is too long for you, show selected 𝑤 for continuous-valued, binary-valued, and the constant term.\
Case 1: lambda = 1 for all coefficients \
Case 2: lambda = 1 for all but the intercept, no regularization for intercept term. \
Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for intercept term.

* mylogistic_l2 class

In [291]:
class mylogistic_l2:
    
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        self.reg_vec = reg_vec 
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
        
    def Sigmoid(self, alpha):  # 計算Sigmoid value，(P(Y|X))
            return 1 / (1 + np.exp(-alpha))
        
    def fit(self, x, y):  # 找出最佳feature weighting
        if self.add_intercept == True:  # 處理intercept
            self.x = np.hstack((x, np.ones((x.shape[0], 1))))
        self.y = y    
     
        # 1. 初始feature weighting
        w = np.dot(np.linalg.inv(np.dot(self.x.T, self.x) + self.reg_vec.mean() * np.eye(self.x.shape[1])), np.dot(self.x.T, self.y))

        # 2. gradient和hessian
        def Gradient(x, y, w, lmda):
            t = self.Sigmoid(np.dot(x, w))
            return np.dot(x.T, (t - y)) + np.dot(lmda, w)
        
        def Hessian(x, w, lmda):
            t = self.Sigmoid(np.dot(x, w))
            R = np.diag(t * (1 - t))
            return np.dot(np.dot(x.T, R), x) + lmda
        
        # 3. error
        def Error(x, y, w, lmda):
            t = self.Sigmoid(np.dot(self.x, w))
            return np.dot(np.dot(w.T, lmda/2), w) - np.sum(y*np.log(t) + (1 - y*np.log(t)))
        
        print("Initial error:", Error(self.x, self.y, w, self.reg_vec))
        # 4. 運算：只要error進步的值仍大於tol，就執行迴圈，滿足error 小於tol後，儲存w：
        for i in range(self.max_iter):
            error_old = Error(self.x, self.y, w, self.reg_vec)  # 初始w的error值
            w -= np.dot(np.linalg.inv(Hessian(self.x, w, self.reg_vec)), Gradient(self.x, self.y, w, self.reg_vec))
            error_new = Error(self.x, self.y, w, self.reg_vec)
            print("iter round", i+1, "error:", error_new)

            if abs(error_new) < abs(error_old):  # 保留可產生error最小的w
                optimal_w = w
                
            if abs(error_new - error_old) < self.tol:  # error進步幅度過小就停止
                break

        self.w = optimal_w
        return self.w

        
    def predict(self, x_test):
        if self.add_intercept == True:  # 處理intercept
            self.x_test = np.hstack((x_test, np.ones((x_test.shape[0], 1))))
            
        # 1. 計算P(Y|X)，再做二元分類(機率介於0~1，直接四捨五入)，寫成一個array
        t = self.Sigmoid(np.dot(self.x_test, self.w))
        t = t.round()        
        # 2. 回傳該二元分類array
        return t

* Case 1: lambda = 1 for all coefficients 

In [292]:
lambda_vec1 = np.diag(np.ones(x_train.shape[1] + 1))

In [293]:
logic = mylogistic_l2(reg_vec = lambda_vec1, max_iter = 30, tol = 1e-5, add_intercept = True)
logic.fit(x_train, y_train) 
y_pred = logic.predict(x_test)

Initial error: -30151.184662909913
iter round 1 error: -30143.669924741942
iter round 2 error: -30131.20997897284
iter round 3 error: -30118.808974215895
iter round 4 error: -30111.09925965939
iter round 5 error: -30109.103391153676
iter round 6 error: -30108.96163443724
iter round 7 error: -30108.960420891963
iter round 8 error: -30108.960420739604


In [294]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.848688143473929

* Case 2: lambda = 1 for all but the intercept, no regularization for intercept term. 

In [298]:
lambda_vec2 = np.diag(np.hstack((np.ones(x_train.shape[1]), 0)))

In [299]:
logic = mylogistic_l2(reg_vec = lambda_vec2, max_iter = 30, tol = 1e-5, add_intercept = True)
logic.fit(x_train, y_train) 
y_pred = logic.predict(x_test)

Initial error: -30151.19495088686
iter round 1 error: -30148.29712786545
iter round 2 error: -30144.06509453287
iter round 3 error: -30139.807153521666
iter round 4 error: -30136.170487142077
iter round 5 error: -30134.958982917244
iter round 6 error: -30134.862397269226
iter round 7 error: -30134.861454746464
iter round 8 error: -30134.861454592


In [300]:
accuracy_score(y_test, y_pred)

0.848090335436732

* Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for intercept term.

In [295]:
lambda_vec3 = np.diag(np.hstack((np.ones(5), np.array([0.5] * 89), 0)))

In [296]:
logic = mylogistic_l2(reg_vec = lambda_vec3, max_iter = 30, tol = 1e-5, add_intercept = True)
logic.fit(x_train, y_train)
y_pred = logic.predict(x_test)

Initial error: -30151.591304092657
iter round 1 error: -30149.912681528716
iter round 2 error: -30147.332186583728
iter round 3 error: -30144.626070038288
iter round 4 error: -30141.867480138906
iter round 5 error: -30140.496065036386
iter round 6 error: -30140.362384652042
iter round 7 error: -30140.360895329028
iter round 8 error: -30140.360895055914


In [297]:
accuracy_score(y_test, y_pred)

0.8478910660909996

<b>Q1.4 (10%) Further split the training data into subtraining (90%) and tuning (10%) to search for the best hyperparameters. Set the regularization coefficient for the constant term to zero. Allow different regularizations for continuous-valued and binary-valued features. Let 𝑎1 and 𝑎2 denote the regularization coefficients for continuous-valued and binary-valued features. Search the best 𝑎1 and 𝑎2 and report the test accuracy using the best hyper-parameters. You should follow the following procedure to search for the best hyperparameters.
    1. Choose a set of grids among a reasonable range. For example, 10 grids in [0.01, 100].
    2. Conduct grid search with the constraint that 𝑎1 = 𝑎2. Record the best value 𝑎∗1 and 𝑎∗2.
    3. Fix 𝑎1=𝑎∗1, and search 𝑎2 for the best value, call the result the new 𝑎∗2.
    4. Fix  𝑎2=𝑎∗2, and search 𝑎1 for the best value.
    5. Report the selected 𝑎1 and 𝑎2.
    6. Train a model using the selected hyper-parameters, and report the test accuracy.

In [238]:
# Spliting
from sklearn.model_selection import train_test_split
x_subtrain, x_tune, y_subtrain, y_tune = train_test_split(x_train, y_train, test_size = 0.1)

In [308]:
# Grids
gridset = np.random.rand(5)  # 因為hessian的運算速度較慢，因此取5grids
gridset

array([0.97817706, 0.05070542, 0.54873444, 0.23872403, 0.75053172])

* Fix 𝑎1=𝑎∗1, and search 𝑎2 for the best value, call the result the new 𝑎∗2.

In [311]:
optimal_a1 = None
optimal_a2 = None
optimal_accuracy = 0
for i in gridset:
    logic = mylogistic_l2(reg_vec = np.diag([i]*5 + [i]*89 + [0]), max_iter = 1000, tol = 1e-5, add_intercept = True)
    print("a1 = a2 =", i)
    logic.fit(x_subtrain, y_subtrain)
    y_pred = logic.predict(x_tune)
    if accuracy_score(y_pred, y_tune) > optimal_accuracy:
        optimal_accuracy = accuracy_score(y_pred, y_tune)
        optimal_a1 = i
        optimal_a2 = i
print("Optimal lambda for continuous-valued =", optimal_a1)
print("Optimal lambda for binary-valued =", optimal_a2)

a1 = a2 = 0.9781770599951939
Initial error: -27135.470098497797
iter round 1 error: -27132.39206568755
iter round 2 error: -27128.29065182022
iter round 3 error: -27124.028901440146
iter round 4 error: -27120.48566369065
iter round 5 error: -27119.414976716496
iter round 6 error: -27119.335431286872
iter round 7 error: -27119.33474043618
iter round 8 error: -27119.334740348044
a1 = a2 = 0.050705422604084505
Initial error: -27135.971846561595
iter round 1 error: -27135.68198901164
iter round 2 error: -27135.184593444756
iter round 3 error: -27134.550778176093
iter round 4 error: -27133.603266667622
iter round 5 error: -27131.647690284623
iter round 6 error: -27132.79463701689
iter round 7 error: -27132.273933070974
iter round 8 error: -27132.32981681185
iter round 9 error: -27132.328637509345
iter round 10 error: -27132.328637034712
a1 = a2 = 0.5487344441134364
Initial error: -27135.69937151129
iter round 1 error: -27133.802446883376
iter round 2 error: -27131.09031719439
iter round 3 e

* Fix 𝑎1=𝑎∗1, and search 𝑎2 for the best value, call the result the new 𝑎∗2.

In [313]:
optimal_a2 = None
optimal_accuracy = 0
for i in gridset:
    logic = mylogistic_l2(reg_vec = np.diag([optimal_a1]*5 + [i]*89 + [0]), max_iter = 1000, tol = 1e-5, add_intercept = True)
    print("a2 =", i)
    logic.fit(x_subtrain, y_subtrain)
    y_pred = logic.predict(x_tune)
    if accuracy_score(y_pred, y_tune) > optimal_accuracy:
        optimal_accuracy = accuracy_score(y_pred, y_tune)
        optimal_a2 = i
print("Optimal lambda for continuous-valued =", optimal_a1)
print("Optimal lambda for binary-valued =", optimal_a2)

a2 = 0.9781770599951939
Initial error: -27135.46984079232
iter round 1 error: -27132.397962076895
iter round 2 error: -27128.30489495188
iter round 3 error: -27124.052371295063
iter round 4 error: -27120.51333235492
iter round 5 error: -27119.443249255728
iter round 6 error: -27119.36372388253
iter round 7 error: -27119.363033065518
iter round 8 error: -27119.36303297738
a2 = 0.050705422604084505
Initial error: -27135.97178193153
iter round 1 error: -27135.680511470444
iter round 2 error: -27135.181047621998
iter round 3 error: -27134.544738285327
iter round 4 error: -27133.594710412217
iter round 5 error: -27131.634095705056
iter round 6 error: -27132.785802338745
iter round 7 error: -27132.26165589399
iter round 8 error: -27132.318914933643
iter round 9 error: -27132.317681347464
iter round 10 error: -27132.317680831347
a2 = 0.5487344441134364
Initial error: -27135.699359025057
iter round 1 error: -27133.804911145082
iter round 2 error: -27131.096249458315
iter round 3 error: -27128.

* Fix  𝑎2=𝑎∗2, and search 𝑎1 for the best value.

In [314]:
optimal_a1 = None
optimal_accuracy = 0
for i in gridset:
    logic = mylogistic_l2(reg_vec = np.diag([i]*5 + [optimal_a2]*89 + [0]), max_iter = 1000, tol = 1e-5, add_intercept = True)
    print("a1 =", i)
    logic.fit(x_subtrain, y_subtrain)
    y_pred = logic.predict(x_tune)
    if accuracy_score(y_pred, y_tune) > optimal_accuracy:
        optimal_accuracy = accuracy_score(y_pred, y_tune)
        optimal_a1 = i
print("Optimal lambda for continuous-valued =", optimal_a1)
print("Optimal lambda for binary-valued =", optimal_a2)

a1 = 0.9781770599951939
Initial error: -27135.867980435578
iter round 1 error: -27134.9189439075
iter round 2 error: -27133.466608461826
iter round 3 error: -27131.772355395373
iter round 4 error: -27129.671145508863
iter round 5 error: -27127.897704165036
iter round 6 error: -27127.765496034113
iter round 7 error: -27127.761768189546
iter round 8 error: -27127.761763857263
a1 = 0.050705422604084505
Initial error: -27135.868161728347
iter round 1 error: -27134.926257336865
iter round 2 error: -27133.484163388293
iter round 3 error: -27131.801969358345
iter round 4 error: -27129.707799305936
iter round 5 error: -27127.93392596735
iter round 6 error: -27127.8030145006
iter round 7 error: -27127.79928967119
iter round 8 error: -27127.799285337067
a1 = 0.5487344441134364
Initial error: -27135.868064450875
iter round 1 error: -27134.922314082556
iter round 2 error: -27133.474697365564
iter round 3 error: -27131.78599873551
iter round 4 error: -27129.688028401906
iter round 5 error: -27127.9

* 如上所示，我選擇模型lambda參數為連續型變數0.978、二元變數0.238

* Train a model using the selected hyper-parameters, and report the test accuracy.

In [327]:
logic = mylogistic_l2(reg_vec = np.diag([i]*5 + [optimal_a2]*89 + [optimal_a1]), max_iter = 1000, tol = 1e-5, add_intercept = True)
logic.fit(x_train, y_train)
y_pred = logic.predict(x_test)
print("Accuracy score with hyper parameter:", accuracy_score(y_pred, y_test))

Initial error: -30151.793257464287
iter round 1 error: -30147.427157154445
iter round 2 error: -30139.433150392284
iter round 3 error: -30130.76570718343
iter round 4 error: -30124.778304556654
iter round 5 error: -30122.310200312695
iter round 6 error: -30121.954901470195
iter round 7 error: -30121.947646305416
iter round 8 error: -30121.947636911125
Accuracy score with hyper parameter: 0.8477582198605115


In [329]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
print("Precision:", precision_score(y_pred, y_test), "recall:", recall_score(y_pred, y_test))
confusion_matrix(y_pred, y_test)

Precision: 0.6037327562888829 recall: 0.7296502124877411


array([[10531,  1465],
       [  827,  2232]])

<b>Q1.5 (20%) Use sklearn.linear_model.LogisticRegression to train and test the model (including hyperparameter tuning). Compare the estimated parameters and test accuracy with those from your own models.

In [347]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty = "l2", tol = 1e-5, max_iter = 1000, fit_intercept = True )
LR.fit(x_train, y_train)
y_pred = LR.predict(x_test)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [349]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.843639986715377

In [350]:
print("Precision:", precision_score(y_pred, y_test), "recall:", recall_score(y_pred, y_test))
confusion_matrix(y_pred, y_test)

Precision: 0.5777657560183933 recall: 0.7292591328098327


array([[10565,  1561],
       [  793,  2136]])

使用sklearn的logistic regression，可以得到準確率約為0.8436，相比自己設計、經過tuning後的模型低了約0.5%，另外看precision rate與recall rate也都較scikit-learn的模型要高出一些，顯示出parameter tuning的重要性。